# DataWarehousing

### Create Datawarehouse based on the designed Schema

In [1]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('Athleteswarehouse.db')
c = conn.cursor()

# Create dimensions tables
c.execute('''CREATE TABLE IF NOT EXISTS PlayerDimension (
                Record_Id INTEGER PRIMARY KEY AUTOINCREMENT,
                Player TEXT,
                Age INTEGER,
                Position TEXT
            )''')

c.execute('''CREATE TABLE IF NOT EXISTS TeamDimension (
                TeamId INTEGER PRIMARY KEY AUTOINCREMENT,
                Team TEXT
            )''')

c.execute('''CREATE TABLE IF NOT EXISTS SeasonDimension (
                SeasonId INTEGER PRIMARY KEY AUTOINCREMENT,
                Season TEXT
            )''')

# Create facts table
c.execute('''CREATE TABLE IF NOT EXISTS Facts (
                Record_Id INTEGER PRIMARY KEY AUTOINCREMENT,
                Played_Id INTERGER,
                Assists INTEGER,
                YellowCards INTEGER,
                RedCards INTEGER,
                PassCompletionRate REAL,
                DistanceCovered REAL,
                Sprints INTEGER,
                ShotsOnTarget INTEGER,
                TacklesWon INTEGER,
                CleanSheets INTEGER,
                PlayerFatigue REAL,
                MatchPressure REAL,
                InjuryHistory TEXT,
                TrainingHours INTEGER,
                Goals_winsorized INTEGER,
                Height_winsorized REAL,
                Weight_winsorized REAL,
                FatigueInjuryCorrelation_winsorized REAL,
                PressurePerformanceImpact_winsorized REAL,
                EffectiveTraining_winsorized REAL,
                PlayerId INTEGER,
                TeamId INTEGER,
                SeasonId INTEGER,
                FOREIGN KEY(PlayerId) REFERENCES PlayerDimension(Id),
                FOREIGN KEY(TeamId) REFERENCES TeamDimension(TeamId),
                FOREIGN KEY(SeasonId) REFERENCES SeasonDimension(SeasonId)
            )''')

# Commit changes and close connection
conn.commit()
conn.close()


### Connecting to mysql database

In [2]:
import mysql.connector

# Connect to the MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="Sports"
)

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Your warehouse schema creation code goes here

# Remember to commit changes and close the connection when done
conn.commit()
conn.close()


### Adding the sql database to the warehouse

In [3]:
import mysql.connector
import sqlite3

# Connect to MySQL database
mysql_conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="Sports"
)
mysql_cursor = mysql_conn.cursor()

# Connect to SQLite database (AthleteWarehouse.db)
sqlite_conn = sqlite3.connect('AthleteWarehouse.db')
sqlite_cursor = sqlite_conn.cursor()

# Fetch data from MySQL database
mysql_cursor.execute("SELECT * FROM sportsperson_analytics")
data_to_insert = mysql_cursor.fetchall()

# Create tables in SQLite database based on your schema
sqlite_cursor.execute('''CREATE TABLE IF NOT EXISTS PlayerDimension (
                PlayerId INTEGER PRIMARY KEY AUTOINCREMENT,
                Player TEXT,
                Age INTEGER,
                Position TEXT
            )''')

sqlite_cursor.execute('''CREATE TABLE IF NOT EXISTS TeamDimension (
                TeamId INTEGER PRIMARY KEY AUTOINCREMENT,
                Team TEXT
            )''')

sqlite_cursor.execute('''CREATE TABLE IF NOT EXISTS SeasonDimension (
                SeasonId INTEGER PRIMARY KEY AUTOINCREMENT,
                Season TEXT
            )''')

sqlite_cursor.execute('''CREATE TABLE IF NOT EXISTS Facts (
                Record_Id INTEGER PRIMARY KEY AUTOINCREMENT,
                Played_Id INTEGER,
                Player TEXT,
                Team TEXT,
                Age INTEGER,
                Position TEXT,
                Assists INTEGER,
                YellowCards INTEGER,
                RedCards INTEGER,
                PassCompletionRate REAL,
                DistanceCovered REAL,
                Sprints INTEGER,
                ShotsOnTarget INTEGER,
                TacklesWon INTEGER,
                CleanSheets INTEGER,
                PlayerFatigue REAL,
                MatchPressure INTEGER,
                InjuryHistory INTEGER,
                TrainingHours INTEGER,
                Season INTEGER,
                Goals_winsorized REAL,
                Height_winsorized REAL,
                Weight_winsorized REAL,
                FatigueInjuryCorrelation_winsorized REAL,
                PressurePerformanceImpact_winsorized REAL,
                EffectiveTraining_winsorized REAL,
                Last_Updated DATETIME
            )''')

# Insert data into SQLite database
for row in data_to_insert:
    sqlite_cursor.execute('''INSERT INTO Facts (Record_Id, Played_Id, Player, Team, Age, Position, Assists, YellowCards, RedCards,
                            PassCompletionRate, DistanceCovered, Sprints, ShotsOnTarget, TacklesWon, CleanSheets,
                            PlayerFatigue, MatchPressure, InjuryHistory, TrainingHours, Season, Goals_winsorized,
                            Height_winsorized, Weight_winsorized, FatigueInjuryCorrelation_winsorized,
                            PressurePerformanceImpact_winsorized, EffectiveTraining_winsorized, Last_Updated)
                            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', row)

# Commit changes and close connections
sqlite_conn.commit()
sqlite_conn.close()
mysql_conn.close()


## Checking for Complex Analytical Queries

In [4]:
import sqlite3

# Connect to SQLite database (AthleteWarehouse.db)
conn = sqlite3.connect('AthleteWarehouse.db')
cursor = conn.cursor()

# Supporting Complex Analytical Queries:
# Example: Use indexing on frequently used columns
cursor.execute('CREATE INDEX IF NOT EXISTS idx_Player ON Facts(Player)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_Team ON Facts(Team)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_Season ON Facts(Season)')

# # Example: Use window functions for analytical queries
# cursor.execute('''
#     SELECT Player, Team, Season, 
#            AVG(PassCompletionRate) OVER (PARTITION BY Team) AS AvgPassCompletionRate
#     FROM Facts
# ''')



## Implementing Data Security and Access Control:

In [5]:

# Creatin a user table to manage access
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Users (
        Username TEXT PRIMARY KEY,
        Password TEXT,
        Role TEXT
    )
''')

#  Insert users and their roles(just for example)
cursor.execute("INSERT INTO Users (Username, Password, Role) VALUES (?, ?, ?)", ('X', 'password123', 'analyst'))
cursor.execute("INSERT INTO Users (Username, Password, Role) VALUES (?, ?, ?)", ('Y', 'pass123', 'admin'))


#  Authenticating users
def authenticate(username, password):
    cursor.execute("SELECT * FROM Users WHERE Username = ? AND Password = ?", (username, password))
    return cursor.fetchone() is not None

# Example: Perform action based on user role
def perform_action(username, action):
    cursor.execute("SELECT Role FROM Users WHERE Username = ?", (username,))
    role = cursor.fetchone()[0]
    
    if role == 'analyst' and action == 'run_query':
        print("Running complex analytical query...")
        # Run complex analytical query
        
    elif role == 'admin' and action == 'manage_users':
        print("Managing users...")
        # Perform user management tasks

# Authenticate and perform actions
username = input("Enter username: ")
password = input("Enter password: ")

if authenticate(username, password):
    action = input("Enter action (run_query / manage_users): ")
    perform_action(username, action)
else:
    print("Invalid username or password")

# Commit changes and close connection
conn.commit()
# conn.close()
